##### Business problem:
- This notebook is for fetching data from Google Search Console monthly metrics.
- **This notebook runs on first day of each month to get data of previous month**

##### Steps: <br>
- Authentication -> service account
- Campaign performance query
- Transform pulled data
- Append in delta

In [0]:
%run ./gsc_api_authentication

In [0]:
# payload to get keywords level performance
startdate, enddate = dates_calculation()

# replace with your own sitename
siteUrl = "https://www.yoursitename/"

payload = {
    "startDate": startdate,
    "endDate": enddate,
    "dimensions": ["DATE"],
    "rowLimit": 25000,
    "searchType": "WEB"}

#### Evaluate request content

In [0]:
def transform_data(siteUrl, scopes, service_account_path):
    '''
    This function post request and fetch data from google search console and transform data into normarlized view
    Args: 
        siteUrl: str
        scopes: str
        service_account_path: str
    Returns:
        df: pandads dataframe
    '''
    
    siteUrl = siteUrl
    service_account_path = service_account_path
    
    # create response
    response = post_request(siteUrl, scopes, service_account_path, payload) 
    
    # normalize json to pandas
    df = pd.json_normalize(json.loads(response)["rows"])
    
    
    # add reporting month
    df['updated'] = date.today()
    df['month'] = df['keys'].apply(lambda x: x[0])

    df = df.drop(columns='keys')
    
    # validate if duplicates in month
    if df.duplicated(subset="month").sum() == 0:
        return df
    else:
        return None
        print("There are duplicates in dataset campaign, check on that")

In [0]:
df = transform_data(siteUrl, scopes, service_account_path)

#### Write into DBFS - Ingestion

In [0]:
# your database and table name
yourdatabase = "replacedatabasename"
yourtablename = "replacetablename"

In [0]:
from datetime import date
def ingest_data(df):
    '''
    This function append dataframe into delta lake
    Args:
        df: pandas dataframe
    Returns:
        none
    '''

    # create spark dataframe and save to delta
    sdf = spark.createDataFrame(df)

    sdf.write.format("delta") \
        .mode("append") \
        .saveAsTable(f"{yourdatabase}.{yourtablename}")

In [0]:
ingest_data(df=df)